In [1]:
import numpy as np
from ipywidgets import FloatSlider, IntSlider, SelectionSlider
from scipy.integrate import solve_ivp

import plotlymath as pm
from myutils import interact

In [2]:
def frenet_integrate(kappa, tau, s_values, **options):
    TNB_0 = options.pop("TNB_0", np.identity(3))
    alpha_0 = options.pop("alpha_0", np.zeros(3))
    assert(len(s_values.shape) == 1)
    assert(TNB_0.shape == (3, 3))
    assert(alpha_0.shape == (3,))

    def frenet_serret(s, state):
        k = kappa(s)
        t = tau(s)
        frenet_matrix = np.array((
            ( 0,  k,  0), 
            (-k,  0,  t), 
            ( 0, -t,  0), 
        ))
        return ( frenet_matrix @ state.reshape((3, 3)) ).flatten()

    BIG_I = np.identity(3).reshape(3, 3, 1, 1)
    def jacobian(s, state):
        k = kappa(s)
        t = tau(s)
        frenet_matrix = np.array((
            ( 0,  k,  0), 
            (-k,  0,  t), 
            ( 0, -t,  0), 
        ))
        return (frenet_matrix * BIG_I).transpose((2, 0, 3, 1)).reshape((9, 9))

    # Solve the ODE to get T, N, And B as functions of s
    #solve_options = {"method": "LSODA", "jac": jacobian, "dense_output": True, 
    #                 "atol": 1e-10, "rtol": 1e-6}
    solve_options = {"method": "DOP853", "dense_output": True, "atol": 1e-10, "rtol": 1e-6}
    s_range = (s_values[0], s_values[-1])
    solution = solve_ivp(frenet_serret, s_range, TNB_0.flatten(), **solve_options)
    TNB = solution.sol(s_values)
    # Perform composite Simpson's rule on T to get position at each s value
    simpson = TNB[:3].copy()
    T_mid = solution.sol((s_values[:-1] + s_values[1:]) / 2)[:3]
    simpson[:,1:] += simpson[:,:-1] + 4*T_mid
    simpson[:,1:] *= (s_values[1:] - s_values[:-1]) / 6
    simpson[:,0] = alpha_0
    return ( np.cumsum(simpson, axis=-1).T, TNB.T.reshape((-1, 3, 3)) )


In [3]:
def frenet_close_curve(kappa, tau, param_min, param_max, s_values, **options):
    """Approximate value of parameter that makes a closed curve

    kappa and tau should have signature kappa(a, s), where "a" is a parameter
    """
    accuracy = options.pop("accuracy", 1e-6)
    TNB_0 = options.setdefault("TNB_0", np.identity(3))
    alpha_0 = options.setdefault("alpha_0", np.zeros(3))
    N0 = TNB_0[1]
    B0 = TNB_0[2]
    param = param_min
    vector_min = None
    vector_max = None
    def k(s): return kappa(param, s)
    def t(s): return tau(param, s)
    while abs(param_max - param_min) > accuracy:
        alpha, TNB = frenet_integrate(k, t, s_values, **options)
        vector = alpha[-1] - alpha_0
        if vector_min is None:
            vector_min = vector
            param = param_max
        elif vector_max is None:
            vector_max = vector
            param = (param_min + param_max) / 2
        else:
            a = vector @ vector_min
            b = vector @ vector_max
            if a < 0 and b > 0:
                #print(f"Closer to max!")
                #print(f"    {vector} is closer to {vector_max} ({b})")
                #print(f"    than to {vector_min} ({a})")
                param_max = param
                vector_max = vector
            elif a > 0 and b < 0:
                #print(f"Closer to min!")
                #print(f"    {vector} is closer to {vector_min} ({a})")
                #print(f"    than to {vector_max} ({b})")
                param_min = param
                vector_min = vector
            else:
                print(f"Can't continue!")
                print(f"    Current vector = {vector}")
                print(f"    Dot product with vector_min={vector_min} is {a}")
                print(f"    Dot product with vector_max={vector_max} is {b}")
                break
            #print(f"    New range: {param_min} ≤ a ≤ {param_max}")
            param = (param_min + param_max) / 2
    return param


In [4]:
def frenet_interactive(kappa, tau, s_values, **options):
    TNB_0 = options.setdefault("TNB_0", np.identity(3))
    alpha_0 = options.setdefault("alpha_0", np.zeros(3))
    figure, plot = pm.make_figure(widget=True, specs=[[{"type": "scene"}]])
    plot.axes_labels(*options.pop("axes_labels", ("x", "y", "z")))
    alpha, TNB = frenet_integrate(kappa, tau, s_values)

    alpha_min = np.min(alpha, axis=0)
    alpha_max = np.max(alpha, axis=0)
    alpha -= (alpha_max + alpha_min) / 2
    xmax = 0.5 * (alpha_max - alpha_min).max()
    xmax = max(xmax + 1, xmax * 1.2)
    plot.axes_ranges((-xmax, xmax), (-xmax, xmax), (-xmax, xmax), scale=(1, 1, 1))
    plot.lines3d(alpha, name="Curve", id="Curve")

    @interact(s=SelectionSlider(options=s_values, description="$s$"))
    def update(s):
        i = np.argmin(np.abs(s_values - s))
        with figure.batch_update():
            plot.points3d([alpha[i]], size=4, color="black", name="Point", id="Point")
            plot.vector3d(TNB[i,0], start=alpha[i], color="green", name="T", id="T")
            plot.vector3d(TNB[i,1], start=alpha[i], color="red",   name="N", id="N")
            plot.vector3d(TNB[i,2], start=alpha[i], color="gold",  name="B", id="B")

    plot["Point"].update(visible="legendonly")
    for v in ("T", "N", "B"):
        for trace in plot[v]:
            trace.update(visible="legendonly")
    return figure


In [5]:
def parameter_interactive(kappa, tau, s_maxes, param_range, **options):
    samples = options.pop("samples", 4001)
    TNB_0 = options.setdefault("TNB_0", np.identity(3))
    alpha_0 = options.setdefault("alpha_0", np.zeros(3))
    figure, plot = pm.make_figure(widget=True, specs=[[{"type": "scene"}]])
    plot.axes_labels(*options.pop("axes_labels", ("x", "y", "z")))

    amin, amax, avalue, alabel = param_range
    @interact(a=FloatSlider(min=amin, max=amax, value=avalue, 
                            step=(amax - amin) / 200, description=alabel), 
              s_max=SelectionSlider(options=s_maxes, description="$s_{max}$"), 
              pos=FloatSlider(min=0, max=1, value=0, step=0.01, description="position"))
    def update(a, s_max, pos):
        def k(s): return kappa(a, s)
        def t(s): return tau(a, s)
        s_values = np.linspace(0, s_max, samples)
        alpha, TNB = frenet_integrate(k, t, s_values, **options)
        alpha_min = np.min(alpha, axis=0)
        alpha_max = np.max(alpha, axis=0)
        alpha -= (alpha_max + alpha_min) / 2
        xmax = 0.5 * (alpha_max - alpha_min).max()
        xmax = max(xmax + 1, xmax * 1.2)
        i = round(pos * (samples - 1))
        with figure.batch_update():
            plot.axes_ranges((-xmax, xmax), (-xmax, xmax), (-xmax, xmax), scale=(1, 1, 1))
            plot.lines3d(alpha, name="Curve", id="Curve")
            plot.points3d([alpha[i]], size=4, color="black", name="Point", id="Point")
            plot.vector3d(TNB[i,0], start=alpha[i], color="green", name="T", id="T")
            plot.vector3d(TNB[i,1], start=alpha[i], color="red",   name="N", id="N")
            plot.vector3d(TNB[i,2], start=alpha[i], color="gold",  name="B", id="B")

    plot["Point"].update(visible="legendonly")
    for v in ("T", "N", "B"):
        for trace in plot[v]:
            trace.update(visible="legendonly")
    return figure


In [6]:
# Testing: This should give a circle, of radius 3
def kappa(s): return 1/3
def tau(s): return 0
s_values = np.linspace(0, 30, 3001)
frenet_interactive(kappa, tau, s_values)

interactive(children=(SelectionSlider(description='$s$', options=(0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.0…

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Curve',
              'scene': 'scene',
 …

In [7]:
# Testing: This should give a helix, of radius 3
def kappa(s): return 3/np.sqrt(10)
def tau(s): return 1/np.sqrt(10)
s_values = np.linspace(0, 30, 3001)
frenet_interactive(kappa, tau, s_values)

interactive(children=(SelectionSlider(description='$s$', options=(0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.0…

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Curve',
              'scene': 'scene',
 …

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>


## Here we can see how a helix changes as we change its torsion

### (keeping the curvature constant at $\kappa = 1$)


In [8]:
def kappa(a, s): return 1
def tau(a, s): return a
s_maxes = [(fr"{n}π", n*np.pi) for n in range(2, 7, 2)] + [("A lot", 30.0)]
parameter_interactive(kappa, tau, s_maxes, (-0.5, 1.5, 0, "torsion"))

interactive(children=(FloatSlider(value=0.0, description='torsion', max=1.5, min=-0.5, step=0.01), SelectionSl…

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Curve',
              'scene': 'scene',
 …

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>


## This is where things get interesting...

### What does a curve with constant curvature and periodic torsion look like? 

In [9]:
def kappa(a, s): return 1
def tau(a, s): return a * np.cos(s)
s_maxes = [(fr"{n}π", n*np.pi) for n in range(2, 13, 2)] + [("A lot", 200.0)]
parameter_interactive(kappa, tau, s_maxes, (0, 2, 0, "Amplitude"))

interactive(children=(FloatSlider(value=0.0, description='Amplitude', max=2.0, step=0.01), SelectionSlider(des…

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Curve',
              'scene': 'scene',
 …

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>


## Below we solve for (er... numerically approximate) the amplitude of the cosine function that makes the curve a perfect closed loop. 

### Starting with a period of $4\pi$ (the “baseball seam” curve)

In [10]:
def kappa(a, s): return 1
def tau(a, s): return a * np.cos(s)
s_values = np.linspace(0, 4*np.pi, 4001)
magic_a = frenet_close_curve(kappa, tau, 1, 1.02, s_values)
print(f"Magic value of parameter is {magic_a}")

Magic value of parameter is 1.0084475708007812


In [11]:
def kappa(s): return 1
def tau(s): return magic_a * np.cos(s)
s_values = np.linspace(0, 4*np.pi, 2001)
frenet_interactive(kappa, tau, s_values)

interactive(children=(SelectionSlider(description='$s$', options=(0.0, 0.006283185307179587, 0.012566370614359…

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Curve',
              'scene': 'scene',
 …

<br>
<br>
<br>
<br>
<br>

### Next with a period of $6\pi$ (three oscillations as it goes around the loop)

In [12]:
def kappa(a, s): return 1
def tau(a, s): return a * np.cos(s)
s_values = np.linspace(0, 6*np.pi, 12001)
magic_a = frenet_close_curve(kappa, tau, 1.2, 1.4, s_values)
print(f"Magic value of parameter is {magic_a}")

Magic value of parameter is 1.3547428131103514


In [13]:
def kappa(s): return 1
def tau(s): return magic_a * np.cos(s)
s_values = np.linspace(0, 6*np.pi, 12001)
frenet_interactive(kappa, tau, s_values)

interactive(children=(SelectionSlider(description='$s$', options=(0.0, 0.0015707963267948967, 0.00314159265358…

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Curve',
              'scene': 'scene',
 …

<br>
<br>
<br>
<br>
<br>

### Next with a period of $8\pi$ (four oscillations as it goes around the loop)

In [14]:
def kappa(a, s): return 1
def tau(a, s): return a * np.cos(s)
s_values = np.linspace(0, 8*np.pi, 16001)
magic_a = frenet_close_curve(kappa, tau, 1.4, 1.6, s_values)
print(f"Magic value of parameter is {magic_a}")

Magic value of parameter is 1.5318492889404296


In [15]:
def kappa(s): return 1
def tau(s): return magic_a * np.cos(s)
s_values = np.linspace(0, 8*np.pi, 16001)
frenet_interactive(kappa, tau, s_values)

interactive(children=(SelectionSlider(description='$s$', options=(0.0, 0.0015707963267948967, 0.00314159265358…

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Curve',
              'scene': 'scene',
 …

<br>
<br>
<br>
<br>
<br>

### Next with a period of $10\pi$ (five oscillations as it goes around the loop)

In [16]:
def kappa(a, s): return 1
def tau(a, s): return a * np.cos(s)
s_values = np.linspace(0, 10*np.pi, 20001)
magic_a = frenet_close_curve(kappa, tau, 1.6, 1.7, s_values)
print(f"Magic value of parameter is {magic_a}")

Magic value of parameter is 1.6398418426513675


In [17]:
def kappa(s): return 1
def tau(s): return magic_a * np.cos(s)
s_values = np.linspace(0, 10*np.pi, 20001)
frenet_interactive(kappa, tau, s_values)

interactive(children=(SelectionSlider(description='$s$', options=(0.0, 0.0015707963267948967, 0.00314159265358…

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Curve',
              'scene': 'scene',
 …

<br>
<br>
<br>
<br>
<br>

### Next with a period of $12\pi$ (six oscillations as it goes around the loop)

In [18]:
def kappa(a, s): return 1
def tau(a, s): return a * np.cos(s)
s_values = np.linspace(0, 12*np.pi, 12001)
magic_a = frenet_close_curve(kappa, tau, 1.7, 1.8, s_values)
print(f"Magic value of parameter is {magic_a}")

Magic value of parameter is 1.712703323364258


In [19]:
def kappa(s): return 1
def tau(s): return magic_a * np.cos(s)
s_values = np.linspace(0, 12*np.pi, 12001)
frenet_interactive(kappa, tau, s_values)

interactive(children=(SelectionSlider(description='$s$', options=(0.0, 0.0031415926535897933, 0.00628318530717…

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Curve',
              'scene': 'scene',
 …

### etc...


<br>
<br>
<br>
<br>
<br>

## What happens if instead of increasing the amplitude of the cosine function (the torsion), we decrease it? 

### With various periods, this can yield other interesting closed loops. They all have some self-intersections, but some interesting shapes appear. 


### First, with a period of $6\pi$ (three full oscillations)

In [20]:
def kappa(a, s): return 1
def tau(a, s): return a * np.cos(s)
s_values = np.linspace(0, 6*np.pi, 12001)
magic_a = frenet_close_curve(kappa, tau, 0.6, 0.7, s_values)
print(f"Magic value of parameter is {magic_a}")

Magic value of parameter is 0.6690608978271483


In [21]:
def kappa(s): return 1
def tau(s): return magic_a * np.cos(s)
s_values = np.linspace(0, 6*np.pi, 12001)
frenet_interactive(kappa, tau, s_values)

interactive(children=(SelectionSlider(description='$s$', options=(0.0, 0.0015707963267948967, 0.00314159265358…

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Curve',
              'scene': 'scene',
 …

<br>
<br>
<br>
<br>
<br>

### Next, with a period of $8\pi$ (four full oscillations)

In [22]:
def kappa(a, s): return 1
def tau(a, s): return a * np.cos(s)
s_values = np.linspace(0, 8*np.pi, 16001)
magic_a = frenet_close_curve(kappa, tau, 0.48, 0.52, s_values)
print(f"Magic value of parameter is {magic_a}")

Magic value of parameter is 0.5009951782226563


In [23]:
def kappa(s): return 1
def tau(s): return magic_a * np.cos(s)
s_values = np.linspace(0, 8*np.pi, 16001)
frenet_interactive(kappa, tau, s_values)

interactive(children=(SelectionSlider(description='$s$', options=(0.0, 0.0015707963267948967, 0.00314159265358…

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Curve',
              'scene': 'scene',
 …

<br>
<br>
<br>
<br>
<br>

### Next, with a period of $10\pi$ (five full oscillations)

In [24]:
def kappa(a, s): return 1
def tau(a, s): return a * np.cos(s)
s_values = np.linspace(0, 10*np.pi, 20001)
magic_a = frenet_close_curve(kappa, tau, 0.38, 0.42, s_values)
print(f"Magic value of parameter is {magic_a}")

Magic value of parameter is 0.4005062866210937


In [25]:
def kappa(s): return 1
def tau(s): return magic_a * np.cos(s)
s_values = np.linspace(0, 10*np.pi, 20001)
frenet_interactive(kappa, tau, s_values)

interactive(children=(SelectionSlider(description='$s$', options=(0.0, 0.0015707963267948967, 0.00314159265358…

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Curve',
              'scene': 'scene',
 …

<br>
<br>
<br>
<br>
<br>

### Next, with a period of $12\pi$ (six full oscillations)

In [26]:
def kappa(a, s): return 1
def tau(a, s): return a * np.cos(s)
s_values = np.linspace(0, 12*np.pi, 12001)
magic_a = frenet_close_curve(kappa, tau, 0.31, 0.35, s_values)
print(f"Magic value of parameter is {magic_a}")

Magic value of parameter is 0.33362518310546874


In [27]:
def kappa(s): return 1
def tau(s): return magic_a * np.cos(s)
s_values = np.linspace(0, 12*np.pi, 12001)
frenet_interactive(kappa, tau, s_values)

interactive(children=(SelectionSlider(description='$s$', options=(0.0, 0.0031415926535897933, 0.00628318530717…

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Curve',
              'scene': 'scene',
 …

<br>
<br>
<br>
<br>
<br>

## Here, just for fun, we try a curve with periodic curvature *and* periodic torsion. 

### There's only one closed curve shown below, but you can find many more. Try playing around and see what you can find. 


In [30]:
def kappa(a, s): return 1 + np.cos(s)
def tau(a, s): return a * np.sin(s)
s_maxes = [(fr"{n}π", n*np.pi) for n in range(2, 13, 2)] + [("A lot", 200.0)]
parameter_interactive(kappa, tau, s_maxes, (0, 2, 0, "Amplitude"))

interactive(children=(FloatSlider(value=0.0, description='Amplitude', max=2.0, step=0.01), SelectionSlider(des…

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Curve',
              'scene': 'scene',
 …

In [28]:
def kappa(a, s): return 1 + np.cos(s)
def tau(a, s): return a * np.sin(s)
s_values = np.linspace(0, 6*np.pi, 60001)
magic_a = frenet_close_curve(kappa, tau, 1.05, 1.07, s_values)
print(f"Magic value of parameter is {magic_a}")

Magic value of parameter is 1.0624880981445313


In [29]:
def kappa(s): return 1 + np.cos(s)
def tau(s): return magic_a * np.sin(s)
s_values = np.linspace(0, 6*np.pi, 6001)
options = dict(axes_ranges=((-10, 10), (-10, 10), (-10, 10)))
frenet_interactive(kappa, tau, s_values, **options)

interactive(children=(SelectionSlider(description='$s$', options=(0.0, 0.0031415926535897933, 0.00628318530717…

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'Curve',
              'scene': 'scene',
 …